<h1> 2c. Monitoring training </h1>

This notebook is Lab2c of CPB 102, Google's course on Machine Learning using Cloud ML.

In this notebook, we demonstrate monitoring the training of a TensorFlow model and visualizing the graph using TensorBoard. 

In [20]:
import datalab.bigquery as bq
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

Code here is same as Lab2a.

In [21]:
FEATURES = ['pickuplon','pickuplat','dropofflon','dropofflat','passengers']
TARGET = 'fare_amount'

def read_dataset(filename):
  columns = list(FEATURES)
  columns.append(TARGET) # in CSV, target is last column, after the features
  # read with Pandas
  df = pd.read_csv(filename, header=None, names=columns)
  # tensorflow prefers float32
  df[:] = df[:].astype('float32')
  # create features, columns
  feature_cols = {k: tf.constant(df[k].values) for k in FEATURES}
  labels = tf.constant(df[TARGET].values)
  return feature_cols, labels

def get_train():
  return read_dataset('../lab1a/taxi-train.csv')

def get_valid():
  return read_dataset('../lab1a/taxi-valid.csv')

def get_test():
  return read_dataset('../lab1a/taxi-test.csv')

<h2> Monitoring tf.learn </h2>

Set up evaluation of validation set.

In [22]:
tf.logging.set_verbosity(tf.logging.INFO)

class ValidationHook(tf.train.SessionRunHook):
    def __init__(self, sess_, model_):
      self.session = sess_
      self.model = model_
    def begin(self):
      stepno = tf.train.get_global_step()
      if (stepno % 50 == 0):
        metrics = self.model.evaluate(input_fn=get_valid, steps=1)
        print 'Step = {} RMSE on {} dataset = {}'.format(stepno, 'validation', np.sqrt(metrics['loss']))

Pass the hook into the model.fit() function

In [23]:
tf.logging.set_verbosity(tf.logging.INFO)
feature_cols = [tf.contrib.layers.real_valued_column(k)
                  for k in FEATURES]
shutil.rmtree('taxi_model', ignore_errors=True) # start fresh each time
model = tf.contrib.learn.LinearRegressor(
      feature_columns=feature_cols, model_dir='taxi_model',)

with tf.Session() as sess:
  hook = ValidationHook(sess, model)
  model.fit(input_fn=get_train, steps=1000, monitors=[hook]) # DOESN'T WORK

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3c8c2f14d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3c8c2f14d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.


...

INFO:tensorflow:loss = 89.0195, step = 901


INFO:tensorflow:Saving checkpoints for 1000 into taxi_model/model.ckpt.


INFO:tensorflow:Saving checkpoints for 1000 into taxi_model/model.ckpt.


INFO:tensorflow:Loss for final step: 89.0195.


INFO:tensorflow:Loss for final step: 89.0195.


Can use tensorboard to look at the training.

In [9]:
%tensorboard start --logdir=taxi_model

In [12]:
%tensorboard stop --pid=4821

Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License